In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load data from output_labels.csv
csv_path = 'D:\\Semester 7\\FYP\\preprocessing\\output_labels.csv'
df = pd.read_csv(csv_path)

# Extract file paths and class labels
file_paths = df['Path'].values
class_labels = df['Class'].values

In [3]:
from scipy.stats import skew, kurtosis
from scipy.signal import find_peaks

ecg_features = []
for path in file_paths:
    # Load ECG data from CSV file
    ecg_df = pd.read_csv(path)
    # Assuming your ECG data is in columns I, II, III, AVR, AVL, AVF, V1, V2, V3, V4, V5, V6
    ecg_values = ecg_df[['I', 'II', 'III', 'AVR', 'AVL',
                         'AVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']].values
    # Extract features from the ECG data
    ecg_mean = np.mean(ecg_values, axis=0)
    ecg_std = np.std(ecg_values, axis=0)
    ecg_median = np.median(ecg_values, axis=0)
    ecg_variance = np.var(ecg_values, axis=0)
    ecg_skewness = skew(ecg_values, axis=0)
    ecg_kurtosis = kurtosis(ecg_values, axis=0)
    ecg_peak_to_peak = np.ptp(ecg_values, axis=0)
    ecg_rms = np.sqrt(np.mean(np.square(ecg_values), axis=0))
    # Concatenate features
    ecg_features.append(np.concatenate([ecg_mean, ecg_std, ecg_median, ecg_variance, ecg_skewness, ecg_kurtosis, ecg_peak_to_peak, ecg_rms]))

X = np.array(ecg_features)
y = np.array(class_labels)

# Optionally, scale the features using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [4]:
print(X.shape, y.shape)

(5000, 96) (5000,)


In [5]:
y

array(['NORM', 'NORM', 'NORM', ..., 'HYP', 'HYP', 'HYP'], dtype=object)

In [6]:
from sklearn.preprocessing import LabelEncoder

In [7]:
# shuufle the data
from sklearn.utils import shuffle
X, y = shuffle(X, y)


In [8]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# No need to reshape X for LSTM input

# Instantiate the encoder
le = LabelEncoder()

# Fit the encoder and transform the labels
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [9]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM , Dropout , Dense
from tensorflow.keras.optimizers import Adam

In [12]:
num_timesteps = X_train.shape[1]  # Assuming each sample is a sequence of length num_timesteps

# Reshape X_train to be 3D: (num_samples, num_timesteps, num_features)
X_train_reshaped = X_train.reshape(X_train.shape[0], num_timesteps, 1)

# Then you can create your LSTM layer like this:

model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(num_timesteps, 1)))
model.add(LSTM(128, return_sequences=False))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.6))  # Add dropout layer for regularization
model.add(Dense(len(np.unique(y_train)), activation='softmax'))

# Compile the model with a lower learning rate and Adam optimizer
optimizer = Adam(learning_rate=0.0001)  # Specify the learning rate here
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])

# Train the model with increased epochs and batch size
model.fit(X_train, y_train, epochs=20, batch_size=32,
          validation_data=(X_test, y_test))

# Evaluate the model and print test accuracy
accuracy = model.evaluate(X_test, y_test)[1]
print(f"Test Accuracy: {accuracy}")

Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 82ms/step - accuracy: 0.2081 - loss: 1.6067 - val_accuracy: 0.2290 - val_loss: 1.5836
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 78ms/step - accuracy: 0.2692 - loss: 1.5663 - val_accuracy: 0.3210 - val_loss: 1.5091
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 78ms/step - accuracy: 0.3296 - loss: 1.5119 - val_accuracy: 0.3160 - val_loss: 1.4955
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 85ms/step - accuracy: 0.3481 - loss: 1.4811 - val_accuracy: 0.3930 - val_loss: 1.4293
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 93ms/step - accuracy: 0.3807 - loss: 1.4364 - val_accuracy: 0.3860 - val_loss: 1.4111
Epoch 6/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 90ms/step - accuracy: 0.3992 - loss: 1.4138 - val_accuracy: 0.4090 - val_loss: 1.3777
Epoch 7/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 90ms/step - accuracy: 0.3931 - loss: 1.3869 - val_accuracy: 0.4020 - val_loss: 1.3755
Epoch 8/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 88ms/step - accuracy: 0.3998 - loss: 1.3916 - 